## Preparing for 4.10
##### I need to adjust the data frames a little so I have what I need for this one.

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#setting file path shortcut
path = r'C:\Users\evras\Desktop\Data Analysis\Immersion\A4 Python\Instacart Basket Analysis'

In [3]:
#importing files
#I'm bringing in a slightly older version, because I need to remerge the df's for what the task needs
df=pd.read_pickle(os.path.join(path,'02 Data', 'Preped Data', 'orders_products_hammon.pkl'))

In [4]:
#making sure user_id is a string
df['user_id']=df['user_id'].astype('str')

In [5]:
df.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,both,mid-range product,Regular Busy,Average Orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,mid-range product,Second Slowest Day,Average Orders
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,mid-range product,Second Slowest Day,Most orders
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,mid-range product,Slowest Day,Average Orders
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,mid-range product,Slowest Day,Most orders


In [6]:
#a few of these I still don't need to time to trim a little fat
df.drop(columns=['add_to_cart_order','reordered','product_name','aisle_id','_merge'],inplace=True)

In [7]:
#since I'm going to need to add state in I'm going to actually do a new version of the customer df.
#I'll get rid of any pii stuff and then we'll be left with the good bits.
df_cust=pd.read_csv(os.path.join(path,'02 Data','Original Data','customers.csv'))

In [8]:
#checking on what I should keep and ditch
df_cust.head(20)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


In [9]:
#if there is more than one value in here I'll keep it
df_cust['date_joined'].value_counts()

9/17/2018     213
2/10/2018     212
4/1/2019      211
9/21/2019     211
12/19/2017    210
             ... 
9/1/2018      141
1/22/2018     140
11/24/2017    139
7/18/2019     138
8/6/2018      128
Name: date_joined, Length: 1187, dtype: int64

In [10]:
#So I need to get rid of names for sure. But I'll also ge rid of gender because while I don't think it really counts as pii it might still be misused
df_cust=df_cust.drop(columns=['First Name','Surnam','Gender'])

In [11]:
df_cust.rename(columns={'STATE':'user_state','Age':'user_age','n_dependants':'user_dependants','fam_status':'user_fam_status', 'income':'user_income'},inplace=True)

In [12]:
#I know user_id is going to need to be a string if I'm merging it with the other one
df_cust['user_id']=df_cust['user_id'].astype('str')

In [13]:
#checking data types and nulls
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   user_id          206209 non-null  object
 1   user_state       206209 non-null  object
 2   user_age         206209 non-null  int64 
 3   date_joined      206209 non-null  object
 4   user_dependants  206209 non-null  int64 
 5   user_fam_status  206209 non-null  object
 6   user_income      206209 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 11.0+ MB


In [14]:
#I'm going to save it as a seperate one jusy because I call on the old one in a different script.
#I think in real life, if it was just one file and one other script, I'd go ahead and update the script
#but for now I want to clearly lable this as the one without any meaningful private info
df_cust.to_pickle(os.path.join(path,'02 Data', 'Preped Data', 'customer_no_pii.pkl'))

In [15]:
#ok, now I need to bring them together
df=df.merge(df_cust,on='user_id')

In [16]:
#checking the columns
df.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,department_id,prices,price_range,busiest_days,busiest_period_of_day,user_state,user_age,date_joined,user_dependants,user_fam_status,user_income
0,2539329,1,1,2,8,0.0,196,7,9.0,mid-range product,Regular Busy,Average Orders,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,7,9.0,mid-range product,Second Slowest Day,Average Orders,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,7,9.0,mid-range product,Second Slowest Day,Most orders,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,7,9.0,mid-range product,Slowest Day,Average Orders,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,7,9.0,mid-range product,Slowest Day,Most orders,Alabama,31,2/17/2019,3,married,40423


In [17]:
#checking the shape, should have 17 columns
df.shape

(32404859, 18)

In [18]:
#doing some quick exploration to just make sure nothing is way out of wack
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 18 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 object 
 2   order_number            int64  
 3   order_day_of_week       int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   department_id           int64  
 8   prices                  float64
 9   price_range             object 
 10  busiest_days            object 
 11  busiest_period_of_day   object 
 12  user_state              object 
 13  user_age                int64  
 14  date_joined             object 
 15  user_dependants         int64  
 16  user_fam_status         object 
 17  user_income             int64  
dtypes: float64(2), int64(9), object(7)
memory usage: 4.6+ GB


In [19]:
df.describe()

,order_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,department_id,prices,user_age,user_dependants,user_income
count,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.239973e+07,3.240486e+07,3.240486e+07,3.240486e+07
mean,1.710745e+06,1.714230e+01,2.738867e+00,1.342515e+01,1.039267e+01,2.559866e+04,9.919792e+00,7.790994e+00,4.946527e+01,1.501896e+00,9.943773e+04
std,9.872988e+05,1.753532e+01,2.090077e+00,4.246380e+00,8.917825e+00,1.408400e+04,6.281485e+00,4.241809e+00,1.848558e+01,1.118865e+00,4.305727e+04
min,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.800000e+01,0.000000e+00,2.590300e+04
25%,8.559470e+05,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00,1.354400e+04,4.000000e+00,4.200000e+00,3.300000e+01,1.000000e+00,6.700400e+04
50%,1.711049e+06,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00,2.530200e+04,9.000000e+00,7.400000e+00,4.900000e+01,2.000000e+00,9.661800e+04
75%,2.565499e+06,2.400000e+01,5.000000e+00,1.600000e+01,1.400000e+01,3.794700e+04,1.600000e+01,1.130000e+01,6.500000e+01,3.000000e+00,1.279120e+05
max,3.421083e+06,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01,4.968800e+04,2.100000e+01,2.500000e+01,8.100000e+01,3.000000e+00,5.939010e+05


In [20]:
df.isnull().sum()

order_id                     0
user_id                      0
order_number                 0
order_day_of_week            0
order_hour_of_day            0
days_since_prior_order       0
product_id                   0
department_id                0
prices                    5127
price_range                  0
busiest_days                 0
busiest_period_of_day        0
user_state                   0
user_age                     0
date_joined                  0
user_dependants              0
user_fam_status              0
user_income                  0
dtype: int64

In [21]:
#I was expecting the nulls on this one, I'll just drop then since 5k isn't even 1/10 of a percent of the data
df=df[df['prices'].isnull()==False]

In [22]:
#making sure we're clear
df.isnull().sum()

order_id                  0
user_id                   0
order_number              0
order_day_of_week         0
order_hour_of_day         0
days_since_prior_order    0
product_id                0
department_id             0
prices                    0
price_range               0
busiest_days              0
busiest_period_of_day     0
user_state                0
user_age                  0
date_joined               0
user_dependants           0
user_fam_status           0
user_income               0
dtype: int64

In [23]:
df['mean_price']=df.groupby(['user_id'])['prices'].transform(np.mean)

In [24]:
df.loc[df['mean_price']<10,'spending_flag']='low spender'

In [25]:
df.loc[df['mean_price']>=10, 'spending_flag']= 'high spender'

In [26]:
df.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,department_id,prices,price_range,busiest_days,busiest_period_of_day,user_state,user_age,date_joined,user_dependants,user_fam_status,user_income,mean_price,spending_flag
0,2539329,1,1,2,8,0.0,196,7,9.0,mid-range product,Regular Busy,Average Orders,Alabama,31,2/17/2019,3,married,40423,6.367797,low spender
1,2398795,1,2,3,7,15.0,196,7,9.0,mid-range product,Second Slowest Day,Average Orders,Alabama,31,2/17/2019,3,married,40423,6.367797,low spender
2,473747,1,3,3,12,21.0,196,7,9.0,mid-range product,Second Slowest Day,Most orders,Alabama,31,2/17/2019,3,married,40423,6.367797,low spender
3,2254736,1,4,4,7,29.0,196,7,9.0,mid-range product,Slowest Day,Average Orders,Alabama,31,2/17/2019,3,married,40423,6.367797,low spender
4,431534,1,5,4,15,28.0,196,7,9.0,mid-range product,Slowest Day,Most orders,Alabama,31,2/17/2019,3,married,40423,6.367797,low spender


In [27]:
df['spending_flag'].value_counts()

low spender     32280141
high spender      119591
Name: spending_flag, dtype: int64

In [28]:
#cool. Now I'm gonna export THIS just so I don't have to all this all over again
df.to_pickle(os.path.join(path,'02 Data', 'Preped Data', 'opc_no_pii.pkl'))

# Starting a new script
### see DI Task 4.10